In [2]:
import pandas as pd
import numpy as np
import os,sys
import pickle
from collections import Counter
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/antslab/anaconda3/bin/python
3.7.7 (default, Mar 26 2020, 15:48:22) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)


In [3]:
import findspark
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SparkSession
# import pandas as pd
# import numpy as np
from tqdm import tqdm
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()

In [4]:
conf.setAppName("pcapAnalyzer")#pcap_analyzer pcapAnalyzer
conf.setMaster("k8s://https://192.168.50.123:6443")
conf.set("spark.driver.host", "192.168.50.123")
conf.set("spark.executor.instances","2") #2
conf.set("spark.executor.cores","6")
conf.set("spark.driver.memory","30g")
conf.set("spark.kubernetes.container.image", "smileocean/spark-py:spark_3.0.0_preview2_python")
conf.set("spark.kubernetes.namespace", "spark-employee") #spark-employee
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName","spark-employee")#spark-employee
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.execution.arrow.enabled", "true")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("Leo_test").config(conf=conf).getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", True)
# spark
# conf.set("spark.sql.execution.arrow.enabled", True)
import databricks.koalas as ks
ks.options.display.max_rows = 10
ks.set_option('compute.max_rows', None)
ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__)
spark.version

0.31.0


'3.0.0-preview2'

In [5]:
df = pickle.load(open('../../spark_data/data/Leo_Spark_Home/tmp/part_df.pkl','rb'))
df

,city,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,...,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,country
0,None,maxko-hosting.com,[45],[138533],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203422000 CST]",[1578565243.203422000],[210.242.145.4],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
1,None,maxko-hosting.com,[45],[138534],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203554000 CST]",[1578565243.203554000],[210.242.145.36],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
2,None,maxko-hosting.com,[45],[138535],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203610000 CST]",[1578565243.203610000],[210.242.145.37],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
3,None,maxko-hosting.com,[45],[138536],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203639000 CST]",[1578565243.203639000],[210.242.145.66],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
4,None,maxko-hosting.com,[45],[138537],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203658000 CST]",[1578565243.203658000],[210.242.145.58],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214748375605,Tbilisi,magticom.ge,[66],[634758],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.909999000 CST]",[1578535393.909999000],[93.177.147.200],[6],[210.242.148.29],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375606,Tbilisi,magticom.ge,[66],[634759],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910020000 CST]",[1578535393.910020000],[93.177.147.200],[6],[210.242.145.13],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375607,Tbilisi,magticom.ge,[66],[634761],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910031000 CST]",[1578535393.910031000],[93.177.147.200],[6],[210.242.148.62],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375608,Tbilisi,magticom.ge,[66],[634762],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910041000 CST]",[1578535393.910041000],[93.177.147.200],[6],[210.242.148.31],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
